# Spatial fidelity template
Infers a spatial fiedlity map for every individual from a ```.mymridon``` experiment file and saves it as a csv, which can be further analyzed in the optinal part or exported into other software such as RStudio.  
This notebook uses the following:
* the py-myrmidon library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* scipy spatial library ([Documentation](https://docs.scipy.org/doc/scipy/reference/spatial.html))

In [1]:
from datetime import datetime, timedelta  # For convenient handling of time and date

import numpy as np  # Fundamental math library in python.
import pandas as pd  # To create a pandas dataframe, an equivalent to an R dataframe
import py_fort_myrmidon as fm
from scipy.spatial import KDTree

## Calculating spatial fidelity

The code here is the same as in the original code written by Matthias. For convenience sake the different components have been converted into functions that can be used to run over multiple fort-studio experiments

### Defining the grid for the spatial fidelity

A spatial tesselation of hexagons is used to define the sites where the observations of the individuals are counted. A nearest neighbour tree (KDTree) is used with the centers of the hexgons mentioned. For a given set of coordinates, a KDTree can be used to efficiently find the seed that is closest to the input coordinates, here the waypoints of the trajectory. The hexagonal boundaries are not defined explicitly. Simply by placing the hexagonal centers accordingly (see ```method ``` variable), the resulting pattern will be hexagonal or squared. The code in the following cell computes the hexagon centers that seed the KDTree.  
This part should probably only be edited w.r.t. the method and the internal radius.

In [ ]:
def define_kdtree(method, xlim, ylim, r_int):
    if method == "vertical":
        dx = 2 * r_int
        dy = np.sqrt(3) * r_int
    elif method == "horizontal":
        dx = np.sqrt(3) * r_int
        dy = 2 * r_int
    elif method == "square":
        dx = 2 * r_int
        dy = 2 * r_int

    n_x = np.diff(xlim) // dx + 1
    n_y = np.diff(ylim) // dy + 1
    osx = n_x * dx - np.diff(xlim)
    osy = n_y * dy - np.diff(ylim)
    x = np.arange(start=xlim[0] - osx / 2, step=dx, stop=n_x * dx + osx / 2)
    y = np.arange(start=ylim[0] - osy / 2, step=dy, stop=n_y * dy + osy / 2)
    xm, ym = np.meshgrid(x, y)
    if method == "vertical":
        xm[::2] = xm[::2] + r_int
    elif method == "horizontal":
        ym[:, 1::2] = ym[:, 1::2] + r_int

    sites = np.stack([xm.flatten(), ym.flatten()], axis=1)
    kdtree = KDTree(sites)
    return kdtree, sites

### Creating a time range iteration function

In [ ]:
def fm_time_range(start_datetime, end_datetime):
    for n in range(int((end_datetime - start_datetime).days) + 1):
        yield fm.Time(start_datetime + timedelta(n))

### Function to calculate presence in the grid

This function takes the counts of each antID in each of the grid cells that were calculated using the kdtree function

In [ ]:
def calc_counts_kdtree(
    exp, t_start, t_end, kdtree, counts, idxmap, counts_cutoff, col_phase
):
    for t_begin in fm_time_range(t_start, t_end):
        trajectories = fm.Query.ComputeAntTrajectories(
            exp, start=t_begin, end=t_begin.Add(fm.Duration.Parse("24h"))
        )
        for t in trajectories:
            dist, zone_indices = kdtree.query(t.Positions[:, 1:3])
            ind, cts = np.unique(zone_indices, return_counts=True)
            counts[t.Space][idxmap[t.Ant], ind] += cts

    for s in exp.Spaces:
        row_sums = counts[s].sum(axis=1)[:, np.newaxis]
        counts[s] = np.divide(
            counts[s], row_sums, where=row_sums > counts_cutoff
        )  # Normalize per individual
        counts[s][
            np.where(row_sums <= counts_cutoff), :
        ] = 0  # Reset individuals with insufficents total counts
        df = pd.DataFrame(data=counts[s], index=exp.Ants)
        f_count = "NormCounts_{}_Space{}_{}_{}_{}.csv".format(
            col_phase,
            s,
            exp.Spaces[s].Name,
            t_start.strftime("%Y%m%d-%H%M%S"),
            t_end.strftime("%Y%m%d-%H%M%S"),
        )
        print(f_count, " done")
        df.to_csv(f_count)

### Function to run the spatial presence calculation over whole experiments

This is a wrapper function for `calc_counts_kdtree` which runs the function over multiple time ranges

In [2]:
def calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
):
    # Open myrmidon file
    exp = fm.Experiment.Open(myrmidon_path)
    # Get frames from first second of experiment tracking
    identifiedFrames = fm.Query.IdentifyFrames(
        exp,
        start=fm.Time(exp_start),
        end=fm.Time(exp_start).Add(fm.Duration().Parse("1s")),
    )
    # Obtain x and y limits fo frame
    xlim = [0, identifiedFrames[0].Width]
    ylim = [0, identifiedFrames[0].Height]
    # get average ant size to calculate the bin size
    # ant_radius = []
    # for ant in exp.Ants:
    #     ant_radius.append(fm.Query.ComputeMeasurementFor(experiment=exp, antID=ant, measurementTypeID=1)[0].LengthPixel)
    # r_int = np.mean(ant_radius)
    r_int = 300  # Using a general value of 300 to account for cases where the fm.Query.ComputeMeasurementFor throws an error
    # Obtain the kdtree
    kdtree, sites = define_kdtree("vertical", xlim, ylim, r_int)
    # Create a dictionary for holding counts data
    # counts_cutoff = 0 # For hard coding a count cutoff inside the function
    # Create a dictionary with an array full of zeros and shape (number individuals x number sites) for each Space
    counts = {}
    for s in exp.Spaces:
        counts[s] = np.zeros((len(exp.Ants), len(sites)))
    idxmap = dict(
        zip(exp.Ants, range(len(exp.Ants)))
    )  # Maps the ant id to the matrix index
    # Run function to calculate counts over each space for each combination of start and end time
    [
        calc_counts_kdtree(
            exp, t_start, t_end, kdtree, counts, idxmap, counts_cutoff, col_phase
        )
        for t_start, t_end, col_phase in zip(
            start_time_list, end_time_list, col_phase_list
        )
    ]

## Colony 42

In [3]:
# Colony 42
myrmidon_path = (
    "/media/egeorge/Elements/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
)
exp_start = datetime(2022, 4, 27, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 10, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel42_"
phase = [
    "control",
    "exp1",
    "exp2",
    "exp3",
    "exp4",
    "exp5",
    "pre1",
    "pre2",
    "pre3",
    "pre4",
    "pre5",
    "post5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [4]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories:   0%|             | 0/1440 [00:29<?, ?tracked min/s]


NormCounts_Cfel42_control_Space1_Nest_20220501-155400_20220501-215400.csv  done
NormCounts_Cfel42_control_Space2_Forage_20220501-155400_20220501-215400.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:30<?, ?tracked min/s]


NormCounts_Cfel42_exp1_Space1_Nest_20220502-160300_20220502-220300.csv  done
NormCounts_Cfel42_exp1_Space2_Forage_20220502-160300_20220502-220300.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:29<?, ?tracked min/s]


...

NormCounts_Cfel42_pre4_Space1_Nest_20220505-090000_20220505-150000.csv  done
NormCounts_Cfel42_pre4_Space2_Forage_20220505-090000_20220505-150000.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:27<?, ?tracked min/s]


NormCounts_Cfel42_pre5_Space1_Nest_20220506-090000_20220506-150000.csv  done
NormCounts_Cfel42_pre5_Space2_Forage_20220506-090000_20220506-150000.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:23<?, ?tracked min/s]


NormCounts_Cfel42_post5_Space1_Nest_20220506-100000_20220506-160000.csv  done
NormCounts_Cfel42_post5_Space2_Forage_20220506-100000_20220506-160000.csv  done


## Colony 1

In [5]:
# Colony 1
myrmidon_path = (
    "/media/egeorge/Elements/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
)
exp_start = datetime(2022, 5, 31, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 15, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel1_"
phase = [
    "control",
    "exp1",
    "exp2",
    "exp3",
    "exp4",
    "exp5",
    "pre1",
    "pre2",
    "pre3",
    "pre4",
    "pre5",
    "post5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [6]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories:   0%|             | 0/1440 [00:22<?, ?tracked min/s]


NormCounts_Cfel1_control_Space1_forage_20220604-144800_20220604-204800.csv  done
NormCounts_Cfel1_control_Space2_nest_20220604-144800_20220604-204800.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:24<?, ?tracked min/s]


NormCounts_Cfel1_exp1_Space1_forage_20220605-145700_20220605-205700.csv  done
NormCounts_Cfel1_exp1_Space2_nest_20220605-145700_20220605-205700.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:24<?, ?tracked min/s]


...

NormCounts_Cfel1_pre4_Space1_forage_20220608-080000_20220608-140000.csv  done
NormCounts_Cfel1_pre4_Space2_nest_20220608-080000_20220608-140000.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:19<?, ?tracked min/s]


NormCounts_Cfel1_pre5_Space1_forage_20220609-080000_20220609-140000.csv  done
NormCounts_Cfel1_pre5_Space2_nest_20220609-080000_20220609-140000.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:05<?, ?tracked min/s]


NormCounts_Cfel1_post5_Space1_forage_20220610-080000_20220610-140000.csv  done
NormCounts_Cfel1_post5_Space2_nest_20220610-080000_20220610-140000.csv  done


## Colony 54

In [7]:
# Colony 54
myrmidon_path = (
    "/media/egeorge/Elements/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
)
exp_start = datetime(2022, 6, 15, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 6, 19, 14, 25).astimezone(tz=None),
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
    datetime(2022, 6, 24, 14, 31).astimezone(tz=None),
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 25, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel54_"
phase = [
    "control",
    "exp1",
    "exp2",
    "exp3",
    "exp4",
    "exp5",
    "pre1",
    "pre2",
    "pre3",
    "pre4",
    "pre5",
    "post5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [8]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories:   0%|             | 0/1440 [00:28<?, ?tracked min/s]


NormCounts_Cfel54_control_Space1_nest_20220619-142500_20220619-202500.csv  done
NormCounts_Cfel54_control_Space2_forage_20220619-142500_20220619-202500.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:28<?, ?tracked min/s]


NormCounts_Cfel54_exp1_Space1_nest_20220620-143500_20220620-203500.csv  done
NormCounts_Cfel54_exp1_Space2_forage_20220620-143500_20220620-203500.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:26<?, ?tracked min/s]


...

NormCounts_Cfel54_pre4_Space1_nest_20220623-080000_20220623-140000.csv  done
NormCounts_Cfel54_pre4_Space2_forage_20220623-080000_20220623-140000.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:24<?, ?tracked min/s]


NormCounts_Cfel54_pre5_Space1_nest_20220624-080000_20220624-140000.csv  done
NormCounts_Cfel54_pre5_Space2_forage_20220624-080000_20220624-140000.csv  done


Computing ant trajectories:   0%|             | 0/1440 [00:23<?, ?tracked min/s]


NormCounts_Cfel54_post5_Space1_nest_20220625-080000_20220625-140000.csv  done
NormCounts_Cfel54_post5_Space2_forage_20220625-080000_20220625-140000.csv  done
